In [24]:
import torch
import torch.nn 
import torch.optim 
import torchvision.transforms 
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [25]:
image_path_train = r'C:/Users/awwab/OneDrive/Desktop/DevCon 2024/images/train'
image_path_test = r'C:/Users/awwab/OneDrive/Desktop/DevCon 2024/images/test'

In [26]:
#define a simple CNN
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #first convolutional layer
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = torch.nn.ReLU()
        #max pooling layer
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        #second convolutional layer
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = torch.nn.ReLU()
        #max pooling layer
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        #first fully connected layer
        self.fc1 = torch.nn.Linear(64 * 56 * 56, 128)
        self.relu3 = torch.nn.ReLU()
        #second fully connected layer
        self.fc2 = torch.nn.Linear(128, 2)  

    #defines a forward pass of the model
    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

In [27]:
#resizes each image to 224x224, converts to tensor, and normalizes 
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    #numbers below are taken from the pre-computed values of the ImageNet dataset
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [28]:
#load datasets
train_dataset = ImageFolder(root=image_path_train, transform=transform)
test_dataset = ImageFolder(root=image_path_test, transform=transform)


In [29]:
batch_size = 32
#creating the data loaders
#batch and shuffle the data during training
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [30]:
#instant of the CNN class
model = CNN()
criterion = torch.nn.CrossEntropyLoss()
#optimizing the model parameters
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [33]:
#setup for training
num_epochs = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=200704, out_features=128, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [34]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)

    model.eval()
    test_loss = 0.0
    test_corrects = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            test_corrects += torch.sum(preds == labels.data)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    test_accuracy = test_corrects.double() / len(test_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/8, Train Loss: 2.5567, Test Loss: 0.6183, Test Accuracy: 0.7750
Epoch 2/8, Train Loss: 0.6655, Test Loss: 0.5992, Test Accuracy: 0.7250
Epoch 3/8, Train Loss: 0.5424, Test Loss: 0.4358, Test Accuracy: 0.8750
Epoch 4/8, Train Loss: 0.3610, Test Loss: 0.3099, Test Accuracy: 0.8250
Epoch 5/8, Train Loss: 0.2386, Test Loss: 0.1346, Test Accuracy: 0.9500
Epoch 6/8, Train Loss: 0.2068, Test Loss: 0.2163, Test Accuracy: 0.8750
Epoch 7/8, Train Loss: 0.1575, Test Loss: 0.1303, Test Accuracy: 0.9750
Epoch 8/8, Train Loss: 0.1244, Test Loss: 0.2386, Test Accuracy: 0.8750


In [11]:
#load example images for example testing
from PIL import Image
image_good = Image.open(r'C:/Users/awwab/OneDrive/Desktop/DevCon 2024/images/example_good_condition.jpg')
image_bad = Image.open(r'C:/Users/awwab/OneDrive/Desktop/DevCon 2024/images/example_bad_condition.jpg')
#prepare images into image tensors
image_tensor_good = transform(image_good).unsqueeze(0).to(device)
image_tensor_bad = transform(image_bad).unsqueeze(0).to(device)

In [17]:
#evaluation mode
model.eval()

#example using an image of a terrestrial ecosystem in good condition 
with torch.no_grad():
    outputs = model(image_tensor_good)
    _, predicted = torch.max(outputs, 1)

#list of labels that are used to predict
result = ['bad condition', 'good condition']
prediction = result[predicted.item()]

print(f"The image is classified as a terrestrial ecosystem that is in {prediction}.")

The image is classified as a terrestrial ecosystem that is in good condition.


In [18]:
##example using an image of a terrestrial ecosystem in bad condition 
with torch.no_grad():
    outputs = model(image_tensor_bad)
    _, predicted = torch.max(outputs, 1)

#list of labels that are used to predict
result = ['bad condition', 'good condition']
prediction = result[predicted.item()]

print(f"The image is classified as a terrestrial ecosystem that is in {prediction}.")

The image is classified as a terrestrial ecosystem that is in bad condition.
